In [1]:
import os
import pickle

pkl_file = 'data/nuscenes-mini/nuscenes_infos_val.pkl'

with open(pkl_file, 'rb') as f:
    temporal_infos = pickle.load(f)

temporal_infos.keys()

dict_keys(['infos', 'metadata'])

In [9]:
temporal_infos['infos'][0].keys()

dict_keys(['lidar_path', 'token', 'sweeps', 'cams', 'lidar2ego_translation', 'lidar2ego_rotation', 'ego2global_translation', 'ego2global_rotation', 'timestamp', 'gt_boxes', 'gt_names', 'gt_velocity', 'num_lidar_pts', 'num_radar_pts', 'valid_flag'])

In [10]:
temporal_infos['infos'][0]['cams']['CAM_FRONT'].keys()

dict_keys(['data_path', 'type', 'sample_data_token', 'sensor2ego_translation', 'sensor2ego_rotation', 'ego2global_translation', 'ego2global_rotation', 'timestamp', 'sensor2lidar_rotation', 'sensor2lidar_translation', 'cam_intrinsic'])

In [11]:
temporal_infos['infos'][0]['lidar2ego_translation']

[0.985793, 0.0, 1.84019]

In [1]:
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils import splits
from PIL import Image
import numpy as np
import cv2
import torch
import mmcv




def convert_egopose_to_matrix_numpy(rotation, translation):
    transformation_matrix = np.zeros((4, 4), dtype=np.float32)
    transformation_matrix[:3, :3] = rotation
    transformation_matrix[:3, 3] = translation
    transformation_matrix[3, 3] = 1.0
    return transformation_matrix

def invert_matrix_egopose_numpy(egopose):
    """ Compute the inverse transformation of a 4x4 egopose numpy matrix."""
    inverse_matrix = np.zeros((4, 4), dtype=np.float32)
    rotation = egopose[:3, :3]
    translation = egopose[:3, 3]
    inverse_matrix[:3, :3] = rotation.T
    inverse_matrix[:3, 3] = -np.dot(rotation.T, translation)
    inverse_matrix[3, 3] = 1.0
    return inverse_matrix

def _get_rot(h):
        return torch.Tensor(
            [
                [np.cos(h), np.sin(h)],
                [-np.sin(h), np.cos(h)],
            ]
        )

def _img_transform(img, resize, resize_dims, crop, flip, rotate):
        ida_rot = torch.eye(2)
        ida_tran = torch.zeros(2)
        # adjust image
       
        
        img = img.resize(resize_dims)
        img = img.crop(crop)
        if flip:
            img = img.transpose(method=Image.FLIP_LEFT_RIGHT)
        img = img.rotate(rotate)

        # post-homography transformation
        ida_rot *= resize
        ida_tran -= torch.Tensor(crop[:2])
        if flip:
            A = torch.Tensor([[-1, 0], [0, 1]])
            b = torch.Tensor([crop[2] - crop[0], 0])
            ida_rot = A.matmul(ida_rot)
            ida_tran = A.matmul(ida_tran) + b
        A = _get_rot(rotate / 180 * np.pi)
        b = torch.Tensor([crop[2] - crop[0], crop[3] - crop[1]]) / 2
        b = A.matmul(-b) + b
        ida_rot = A.matmul(ida_rot)
        ida_tran = A.matmul(ida_tran) + b
        ida_mat = torch.eye(3)
        ida_mat[:2, :2] = ida_rot
        ida_mat[:2, 2] = ida_tran
        return img, ida_mat

val_scenes = splits.mini_val

nusc = NuScenes(version='v1.0-mini', dataroot='data/nuscenes/', verbose=False)

scene_name = 'scene-0103'
scene = next(scene for scene in nusc.scene if scene['name'] == scene_name) 
scene_token = scene['token']

sample = nusc.get('sample', scene['first_sample_token'])

/home/eddy/miniconda3/envs/streampetr/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [273]:

intrinsic_arr = []
extrinsic_arr = []
lidar2img_arr = []
filename_arr = []
img_arr = []

mean = np.array([103.530, 116.280, 123.675], dtype=np.float32)
std = np.array([57.375, 57.120, 58.395], dtype=np.float32)

cams = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']

sd_rec_l = nusc.get('sample_data', sample['data']['LIDAR_TOP'])

cs_record_l = nusc.get('calibrated_sensor', sd_rec_l['calibrated_sensor_token'])

pose_record_l = nusc.get('ego_pose', sd_rec_l['ego_pose_token'])

# ego pose

# cs_record['translation'] -> lidar2ego_translation
# cs_record['rotation'] -> lidar2ego_rotation
# pose_record['translation'] -> ego2global_translation
# pose_record['rotation'] -> ego2global_rotation

e2g_rotation = Quaternion(pose_record_l['rotation']).rotation_matrix
e2g_translation = pose_record_l['translation']
l2e_rotation = Quaternion(cs_record_l['rotation']).rotation_matrix
l2e_translation = cs_record_l['translation']
e2g_matrix = convert_egopose_to_matrix_numpy(e2g_rotation, e2g_translation)
l2e_matrix = convert_egopose_to_matrix_numpy(l2e_rotation, l2e_translation)

ego_pose = e2g_matrix @ l2e_matrix
ego_pose_inv = invert_matrix_egopose_numpy(ego_pose)
timestamp = sd_rec_l['timestamp'] / 1e6




# lidar2img & intrinsic & extrinsic
for cam in cams:

    cam_front_token = sample['data'][cam]

    # 使用 token 获取 sample_data 记录
    sd_rec_c = nusc.get('sample_data', cam_front_token)



    # 获取摄像头的校准数据
    cs_record_c = nusc.get('calibrated_sensor', sd_rec_c['calibrated_sensor_token'])

    pose_record_c = nusc.get('ego_pose', sd_rec_c['ego_pose_token'])



    # lidar
    l2e_t = np.array(cs_record_l['translation'])
    e2g_t = np.array(pose_record_l['translation'])
    l2e_r = np.array(cs_record_l['rotation'])
    e2g_r = np.array(pose_record_l['rotation'])

    # cam
    l2e_t_s = np.array(cs_record_c['translation'])
    e2g_t_s = np.array(pose_record_c['translation'])
    l2e_r_s = np.array(cs_record_c['rotation'])
    e2g_r_s = np.array(pose_record_c['rotation'])


    l2e_r_mat = Quaternion(l2e_r).rotation_matrix
    e2g_r_mat = Quaternion(e2g_r).rotation_matrix

    l2e_r_s_mat = Quaternion(l2e_r_s).rotation_matrix
    e2g_r_s_mat = Quaternion(e2g_r_s).rotation_matrix

    R = (l2e_r_s_mat.T @ e2g_r_s_mat.T) @ (
            np.linalg.inv(e2g_r_mat).T @ np.linalg.inv(l2e_r_mat).T)
    T = (l2e_t_s @ e2g_r_s_mat.T + e2g_t_s) @ (
        np.linalg.inv(e2g_r_mat).T @ np.linalg.inv(l2e_r_mat).T)
    T -= e2g_t @ (np.linalg.inv(e2g_r_mat).T @ np.linalg.inv(l2e_r_mat).T
                    ) + l2e_t @ np.linalg.inv(l2e_r_mat).T

    cam2lidar_r = R.T
    cam2lidar_t = T


    cam2lidar_rt = convert_egopose_to_matrix_numpy(cam2lidar_r, cam2lidar_t)
    lidar2cam_rt = invert_matrix_egopose_numpy(cam2lidar_rt)

    intrinsic = np.array(cs_record_c['camera_intrinsic'])

    viewpad = np.eye(4)
    viewpad[:intrinsic.shape[0], :intrinsic.shape[1]] = intrinsic

    lidar2img_rt = (viewpad @ lidar2cam_rt)

    intrinsic = viewpad
    extrinsic = lidar2cam_rt

    # LoadImage

    img_name = 'data/nuscenes/' + sd_rec_c['filename']
    img_ori = Image.fromarray(mmcv.imread(img_name, 'unchanged'))
  
  
    # ResizeCropFlipRotImage class call

    # img_ori = Image.fromarray(np.uint8(np.zeros((900, 1600, 3))))
    img, ida_mat = _img_transform(
                    img_ori,
                    resize=0.5,
                    resize_dims=(800, 450),
                    crop=(0, 130, 800, 450),
                    flip=False,
                    rotate=0,
                )
    intrinsic[:3, :3] = ida_mat @ intrinsic[:3, :3]

    lidar2img = intrinsic @ extrinsic

    #===================================
    # PadImage

    img = mmcv.impad_to_multiple(np.array(img), 32, pad_val=0)

    # NormalizeImage
    
    img = mmcv.imnormalize(img, mean, std, False)

    intrinsic_arr.append(intrinsic)
    extrinsic_arr.append(extrinsic)
    lidar2img_arr.append(lidar2img)
    filename_arr.append('data/nuscenes/' + sd_rec_c['filename'])
    img_arr.append(img.transpose(2, 0, 1))
    
img_arr = np.stack(img_arr)
# img_arr = np.stack([mmcv.imread(name, 'unchanged') for name in filename_arr], axis=-1)

# padded_img = [mmcv.impad_to_multiple(img,
#                                 32, pad_val=0) for img in img_arr]
print('==intrinsic==\n', intrinsic)
print('==extrinsic==\n', extrinsic)
print('==lidar2img==\n', lidar2img)
print('==ego_pose==\n', ego_pose)

==intrinsic==
 [[624.98146404   0.         412.68840227   0.        ]
 [  0.         624.98146404 101.27408193   0.        ]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]
==extrinsic==
 [[-0.3820134  -0.92305064 -0.04520244  0.08872822]
 [ 0.01385406  0.04318667 -0.998971   -0.29866916]
 [ 0.92405295 -0.38224655 -0.00370989 -0.46464512]
 [ 0.          0.          0.          1.        ]]
==lidar2img==
 [[ 1.42594637e+02 -7.34638261e+02 -2.97817150e+01 -1.36300159e+02]
 [ 1.02241144e+02 -1.17207993e+01 -6.24714065e+02 -2.33719196e+02]
 [ 9.24052954e-01 -3.82246554e-01 -3.70989158e-03 -4.64645118e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
==ego_pose==
 [[-4.8142377e-01  8.7494826e-01  5.1929332e-02  6.0100812e+02]
 [-8.7646019e-01 -4.8009163e-01 -3.6461607e-02  1.6469954e+03]
 [-6.9711814e-03 -6.3067481e-02  9.9798489e-01  1.8232931e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000

In [275]:
import matplotlib.pyplot as plt

# plt.imshow(img_arr[0])
# plt.show()
img_arr.shape

(6, 3, 320, 800)

In [148]:
# ego pose
from pyquaternion import Quaternion
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils import splits

val_scenes = splits.mini_val

nusc = NuScenes(version='v1.0-mini', dataroot='data/nuscenes/', verbose=False)



scene_name = 'scene-0103'
scene = next(scene for scene in nusc.scene if scene['name'] == scene_name) 
scene_token = scene['token']

sample = nusc.get('sample', scene['first_sample_token'])





array([[-4.8142377e-01,  8.7494826e-01,  5.1929332e-02,  6.0100812e+02],
       [-8.7646019e-01, -4.8009163e-01, -3.6461607e-02,  1.6469954e+03],
       [-6.9711814e-03, -6.3067481e-02,  9.9798489e-01,  1.8232931e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)